### Neural Network

Neural Network는 Layer가 두 개 이상인 모형을 의미한다. Layer가 심층적으로 쌓인 경우, Deep Neural Network라고 한다

<center>

![nn.png](Images/neural.png)

</center>

Neural Network에서는 계수(Coefficient)보다는 가중치(Weights)로 더 많이 통용된다. 추정하는 가중치 parameter는 결국 선형모형에서 Coefficient를 추정하는것과 같다.

**활성화 함수(Activation Function)** 는 통계학에서의 연결 함수의 개념을 차용한 것이다. 

대표적인 연결 함수로는 log, sigmoid, logistic이 존재하는데, 이중 sigmoid와 logistic은 초창기 Neural Network model의 활성화 함수로 사용된 것이다

한편, sklearn library에서는 `neural_network.MLPClassifier`, `neural_network.MLPRegressor`를 이용해 Neural Network를 구현할 수 있다

```
sklearn.neural_network.MLPClassifier()
```

#### 1. Input Layers

입력층에서는 데이터의 특성들이 입력된다. 입력 벡터 $\mathbf{x} = [x_1, x_2, ..., x_n]$는 특성 공간에서의 값들을 나타낸다. 입력값 $x_i$는 첫 번째 은닉층으로 전달된다.

$$\mathbf{x} = [x_1, x_2, ..., x_n]$$

#### 2. Hidden Layer

은닉층은 입력값에 가중치(weight)를 곱하고, 그 결과에 **편향(bias)** 을 더한 후 **활성화 함수(activation function)** 를 적용하여 비선형성을 도입한다.

입력 벡터 $\mathbf{x}$가 은닉층으로 전달되며, 가중치 행렬 $\mathbf{W}^{(1)}$와 곱해진다. 이 때, $\mathbf{W}^{(1)}$는 입력층에서 은닉층으로의 가중치 행렬이다. 각 은닉층 노드 $h_j$에서의 계산은 다음과 같다.

$$h_j = f\left( \sum_{i=1}^{n} W_{ji}^{(1)} x_i + b_j^{(1)} \right)$$

여기서 $W_{ji}^{(1)}$는 입력 $x_i$에서 은닉층 뉴런 $h_j$로 가는 가중치, $b_j^{(1)}$는 은닉층 뉴런 $h_j$의 bias(y intercept)이며, $f$는 비선형 활성화 함수(ex. sigmoid, ReLU 등)이다. 통계학에서 활성화함수는 대개 연결 함수를 의미한다. 따라서 은닉층의 출력 벡터 $\mathbf{h}^{(1)}$는 다음과 같다.

$$\mathbf{h}^{(1)} = f\left( \mathbf{W}^{(1)} \mathbf{x} + \mathbf{b}^{(1)} \right)$$

여기서 $\mathbf{W}^{(1)}$는 은닉층의 가중치 행렬, $\mathbf{b}^{(1)}$는 은닉층의 편향 벡터이다.

### 3. Output Layer

출력층도 은닉층과 동일하게 가중치와 편향을 사용하여 계산된다. 은닉층에서 계산된 값들을 출력층으로 전달하여 최종 예측을 생성한다.

은닉층의 출력값 $\mathbf{h}^{(1)}$는 출력층으로 전달된다. 출력층에서 각 출력 노드 $y_k$에 대한 계산은 다음과 같다

$$y_k = f\left( \sum_{j=1}^{m} W_{kj}^{(2)} h_j^{(1)} + b_k^{(2)} \right)$$

여기서 $W_{kj}^{(2)}$는 은닉층 노드 $h_j^{(1)}$에서 출력층 노드 $y_k$로 가는 가중치, $b_k^{(2)}$는 출력층 노드 $y_k$의 bias이고 $f$는 출력층에서 사용하는 활성화 함수이다. 출력층에서의 출력 벡터 $\mathbf{y}$는 다음과 같이 표현된다

$$\mathbf{y} = f\left( \mathbf{W}^{(2)} \mathbf{h}^{(1)} + \mathbf{b}^{(2)} \right)$$

여기서 $\mathbf{W}^{(2)}$는 출력층의 가중치 행렬, $\mathbf{b}^{(2)}$는 출력층의 편향 벡터이다

#### 4. Loss Function

모델이 예측한 값과 실제 값 사이의 차이를 평가하기 위해 손실 함수가 사용된다. 분류 문제에서는 **교차 엔트로피(cross-entropy)** 가 자주 사용되고, 회귀 문제에서는 **평균 제곱 오차(mean squared error, MSE)** 가 주로 사용된다. 교차 엔트로피 손실 함수는 다음과 같다.

$$L(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{k} y_k \log(\hat{y_k})$$

여기서 $y_k$는 실제 클래스 label, $\hat{y_k}$는 모델이 예측한 확률값이다.

#### 5. Backpropagation

신경망 학습의 핵심은 **역전파(backpropagation)** 를 통해 가중치를 업데이트하는 것이다. 역전파는 손실 함수의 기울기를 계산하여 가중치를 업데이트하는 방법이다. 이를 위해 **체인 룰(chain rule)** 을 사용한다. 손실 함수에 대한 가중치의 편미분은 다음과 같다.

$$\frac{\partial L}{\partial W_{ji}} = \frac{\partial L}{\partial y_k} \cdot \frac{\partial y_k}{\partial h_j} \cdot \frac{\partial h_j}{\partial W_{ji}}$$

이를 통해 가중치가 경사 하강법(gradient descent)을 사용해 업데이트된다

$$W_{ji} = W_{ji} - \eta \frac{\partial L}{\partial W_{ji}}$$

여기서 $\eta$는 학습률(learning rate)이다